In [1]:
#load packages
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import pandas as pd
import soundfile as sf


In [2]:
#import data 
path = 'x:/Steering/Recordings/Processed/'
data, fs = sf.read(path + 'ERB=2bil_BF.wav')
classes = pd.read_csv(path + 'Classes_2bil_BF.csv')

In [3]:
X = data[:len(data)-1,:data.shape[1]] 
Y = classes.values[:,0]

In [4]:
len(Y),len(X), X.shape

(889913, 889913, (889913, 18))

In [5]:
#split into training and test set
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X,Y,test_size=0.2,random_state=0)

In [6]:
#do one-hot encoding
from sklearn.preprocessing import OneHotEncoder 
from sklearn.preprocessing import LabelEncoder

enc = OneHotEncoder(sparse=False)

In [7]:
integer_encoded_train = trainY.reshape(len(trainY), 1)
onehot_encoded_train = enc.fit_transform(integer_encoded_train)

integer_encoded_test = testY.reshape(len(testY), 1)
onehot_encoded_test = enc.fit_transform(integer_encoded_test)

In [8]:
onehot_encoded_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]])

In [9]:
#DC removal 
trainX -= (np.mean(trainX, axis=0) + 1e-8)
testX -= (np.mean(testX, axis=0) + 1e-8)

In [10]:
clf = keras.Sequential()

In [11]:
clf.add(keras.layers.Dense(
  units = 10,
  input_dim = trainX.shape[1],   
  activation = 'tanh'
))

clf.add(keras.layers.Dense(
    units = onehot_encoded_train.shape[1],
    input_dim = 10,
    activation = 'softmax'
))


In [12]:
onehot_encoded_train.shape[1],trainX.shape[1]

(2, 18)

In [13]:
clf.compile(loss='mean_squared_error',
    optimizer='Adam',
           metrics=['categorical_accuracy'])

In [15]:
history = clf.fit(trainX,onehot_encoded_train,
                 batch_size = 200, epochs = 10,
                 verbose = 1,
                 validation_split = 0.1)

Train on 640737 samples, validate on 71193 samples
Epoch 1/10
640737/640737 [==============================] - 3s 4us/step - loss: 0.1631 - categorical_accuracy: 0.7444 - val_loss: 0.1636 - val_categorical_accuracy: 0.7437
Epoch 2/10
640737/640737 [==============================] - 3s 4us/step - loss: 0.1630 - categorical_accuracy: 0.7446 - val_loss: 0.1635 - val_categorical_accuracy: 0.7433
Epoch 3/10
640737/640737 [==============================] - 3s 4us/step - loss: 0.1629 - categorical_accuracy: 0.7449 - val_loss: 0.1644 - val_categorical_accuracy: 0.7404
Epoch 4/10
640737/640737 [==============================] - 3s 4us/step - loss: 0.1628 - categorical_accuracy: 0.7449 - val_loss: 0.1633 - val_categorical_accuracy: 0.7435
Epoch 5/10
640737/640737 [==============================] - 3s 5us/step - loss: 0.1626 - categorical_accuracy: 0.7448 - val_loss: 0.1631 - val_categorical_accuracy: 0.7441
Epoch 6/10
640737/640737 [==============================] - 3s 5us/step - loss: 0.1626 - 

In [16]:
testY[1:100], clf.predict_classes(testX[1:100,:])

(array([1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
        1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0], dtype=int64),
 array([0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=int64))

In [17]:
clf.evaluate(testX,onehot_encoded_test,verbose=0)

[0.1623313312188046, 0.7454475989243037]

In [18]:
clf_json = clf.to_json()
with open("NN_OwnVoice_18-10-2_inputs_bilBF.json", "w") as json_file:
    json_file.write(clf_json)
    
clf.save_weights("NN_OwnVoice_18-10-2_inputs_bilBF_weights.h5")

In [19]:
np.save('TestX_NN_OwnVoice_bilateral',testX), np.save('TestY_NN_OwnVoice_bilateral',testY),
np.save('OneHot_testY_bilateral',onehot_encoded_test)